## Load query into a dataframe

In [16]:
from dune_client.client import DuneClient
import pandas as pd
API_KEY = "1EF7SJ7DWZYDkQ0KmbNiutJhtwjQDL5o"
dune = DuneClient(API_KEY)
query_result = dune.get_latest_result(5145525).result.rows #QUERY SQL VISABLE AT https://dune.com/queries/5145525?sidebar=none
print(query_result)
result_table = pd.DataFrame(query_result)
result_table.head()

[{'amount_usd_original': 6006.53039881834, 'amount_usd_poisoning': 0.0015158000000000003, 'block_time_original': '2025-05-01 12:19:21.000 UTC', 'block_time_poisoning': '2025-05-01 12:22:44.000 UTC', 'owner_from_poisoning': 'CDiEyTDCa6r7LFcvfK8RP9rBcDjguQkgWjU5NR3fRGnB', 'owner_to_original': 'CDiE4zviLUSVAvjwVE2mPyjsMDjLjiXNTYSHqC9bF8YB', 'tx_signature_original': '2Af2zLYttvyhmvpAVv2fCpVhX8g8zn3Z19VNPts6niH9', 'tx_signature_poisoning': '5ugh3Wsrq1Z7wto64xVCzANxbuk6f9H6JKwCL62NA7Ay'}, {'amount_usd_original': 1174.17596868, 'amount_usd_poisoning': 0.0015158000000000003, 'block_time_original': '2025-05-01 12:21:04.000 UTC', 'block_time_poisoning': '2025-05-01 12:24:37.000 UTC', 'owner_from_poisoning': 'H4KbyKJvrVh1K2E51ro12wrWBokCqVYrnpf54MyAL1is', 'owner_to_original': 'H4KbFG8bsMhTJc8pCq574H4QehhHBkqf1tqsuRdQ4KMs', 'tx_signature_original': 'G9X7F4JzLzbSGMCndiBdWNi5YzZZakmtkdwq7xS3Q3FE', 'tx_signature_poisoning': '5L6iKj41V2L4oomJvJve1h3aapCijbux7ahXf3CVHQiP'}, {'amount_usd_original': 1336

,amount_usd_original,amount_usd_poisoning,block_time_original,block_time_poisoning,owner_from_poisoning,owner_to_original,tx_signature_original,tx_signature_poisoning
0,6006.530399,0.001516,2025-05-01 12:19:21.000 UTC,2025-05-01 12:22:44.000 UTC,CDiEyTDCa6r7LFcvfK8RP9rBcDjguQkgWjU5NR3fRGnB,CDiE4zviLUSVAvjwVE2mPyjsMDjLjiXNTYSHqC9bF8YB,2Af2zLYttvyhmvpAVv2fCpVhX8g8zn3Z19VNPts6niH9,5ugh3Wsrq1Z7wto64xVCzANxbuk6f9H6JKwCL62NA7Ay
1,1174.175969,0.001516,2025-05-01 12:21:04.000 UTC,2025-05-01 12:24:37.000 UTC,H4KbyKJvrVh1K2E51ro12wrWBokCqVYrnpf54MyAL1is,H4KbFG8bsMhTJc8pCq574H4QehhHBkqf1tqsuRdQ4KMs,G9X7F4JzLzbSGMCndiBdWNi5YzZZakmtkdwq7xS3Q3FE,5L6iKj41V2L4oomJvJve1h3aapCijbux7ahXf3CVHQiP
2,1336.456000,0.001518,2025-05-01 12:04:56.000 UTC,2025-05-01 12:08:29.000 UTC,4p35XMqxbzPB2FEayTW3NdryvwWhSCLJGvHVPMuNVewm,4p35oTCa76VcJWRWgzcSLx8rHMbqrAsohUqRQ9K2jUAm,8YxUm2CgA67gSxu3Vdeij1cEpfbVRurCFHQnUC2sytFz,5L6iKj41V2L4oomJvJve1h3aapCijbux7ahXf3CVHQiP
3,1390.180402,0.001516,2025-05-01 12:29:17.000 UTC,2025-05-01 12:32:40.000 UTC,3E2mC3p9w9igq9gKFZ26JiPA7qvVpKTcN1bcyWRN7VG8,3E2mrKVS7Eetrz3mN8cLxHLY51RWTNz6AJhyAvKbrcT8,27FP2F4uyzJFBbszVVKDatHJZdxFjDiUCgf26FqSuxmg,BLxXRahLDLptsRj1Qep4KFLxxAQebv2V3AdB4PpY9Vxm
4,1519.500000,0.001520,2025-05-01 12:12:25.000 UTC,2025-05-01 12:13:18.000 UTC,7TrJDJMcD4V9ndmQm3D1JBoneytFJo7XJWSCVNMkeQyV,7TrJk16H52kxJiSX2dzb4p5V6g6FiokfxeS4vm2NiJU4,HZLR7hZdnjSqPQvLiYDUgqzR8f4xj91e3vc5aD3xYscM,7TrJDJMcD4V9ndmQm3D1JBoneytFJo7XJWSCVNMkeQyV


## EDA On Poisoning 

In [ ]:
#most common poisoners
common_poisoners = result_table.value_counts("tx_signature_poisoning")
common_poisoners.head()
print(common_poisoners.info)

#get list of transactions to poisoner in next week

#get outgoing transactions from poisoners larger than .01 USD

#volume of transfers from poisoners
attack_volume = result_table.groupby('tx_signature_poisoning')['amount_usd_poisoning'].sum()
attack_volume.sort_values(ascending=False).head()
print(attack_volume.shape)

#most common transfers from poisoner accounts


#get number of successful attacks

#show net amnt earned



<bound method Series.info of tx_signature_poisoning
BLxXRahLDLptsRj1Qep4KFLxxAQebv2V3AdB4PpY9Vxm    161
5L6iKj41V2L4oomJvJve1h3aapCijbux7ahXf3CVHQiP    142
5ugh3Wsrq1Z7wto64xVCzANxbuk6f9H6JKwCL62NA7Ay    121
H2dHTXttztA23BHX4HmTJ1oi5XAzikXusvDppB8a2dqR    120
F6zzYPo8s9QWcqax9asUjo3dCRiEQRUNW5GPkGPqf3RZ     54
                                               ... 
8JtJuWJJjmpa8JkgyA4PSpuH7NcHcthM53TkkPdDhnpa      1
8Jqo5V1ddjTQYRd4srChZ3occabppZWLcq37EtXBGrcy      1
8GyLRyQbAYzHUnfFDaPPudN91qfZaJXsahTD4bBvt8yZ      1
8CfkXbZ7VEuAiB6jnEXyCUVZcwXgbNj5f9eyZTCU6rYB      1
AEnPn8ZMMnkfV7cHLQzozvRoAnxbdQQ8TvhQBBWRSkeW      1
Name: count, Length: 500, dtype: int64>
(500,)
